<a href="https://colab.research.google.com/github/sangho24/sogang/blob/main/EC5320_2024_2_Week10a_Korean_NLP_v1_FOR_STUDENTS_20200572.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EC5320 Week10a codes (Korean NLP)

2024.11.2.<br>

Author: Hyunjoo Yang (hyang@sogang.ac.kr)<br><br>

This notebook uses Keras to do sentimental analysis using Korean texts (Natural language processing).<br><br>

Main source of codes: <br>
https://wikidocs.net/44249 <br><br>

Word Cloud codes are from: <br>
https://datanavigator.tistory.com/37 <br><br>

Data source: <br>
https://github.com/e9t/nsmc/ <br><br>

If you are interested in Korean NLP, please refer to:<br>
https://wikidocs.net/book/2155



# 1. Install modules

In [ ]:
# Korean NLP moddule

!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from tqdm import tqdm

import tensorflow as tf

# 2. Data loading and preprocessing

## 2.1 Upload review data

In [ ]:
# upload review data

!unzip korea_nlp.zip

In [ ]:
train_data_all = pd.read_table('ratings_train.txt')
test_data_all = pd.read_table('ratings_test.txt')

In [ ]:
print('훈련용 리뷰 개수 :',len(train_data_all)) # 훈련용 리뷰 개수 출력

In [ ]:
train_data_all[:10] # 상위 10개 출력

In [ ]:
print('테스트용 리뷰 개수 :',len(test_data_all)) # 테스트용 리뷰 개수 출력

In [ ]:
test_data_all[:10]

## 2.2 Sample both train and test data with n=10,000 each

In [ ]:
train_data = train_data_all.sample(n=10000, random_state=5030)
train_data

In [ ]:
test_data = test_data_all.sample(n=10000, random_state=5030)
test_data

## 2.3. remove English and delete redundant, missing info

In [ ]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

In [ ]:
# document 열의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
print('총 샘플의 수 :',len(train_data))

In [ ]:
train_data['label'].value_counts().plot(kind = 'bar')


In [ ]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

In [ ]:
print(train_data.isnull().values.any())

In [ ]:
print(train_data.isnull().sum())

In [ ]:
train_data.loc[train_data.document.isnull()]

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
print(len(train_data))

In [ ]:
# 알파벳과 공백을 제외하고 모두 제거
eng_text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
print(re.sub(r'[^a-zA-Z ]', '', eng_text))

In [ ]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'] = train_data['document'].replace('', np.nan)
print(train_data.isnull().sum())

In [ ]:
train_data.loc[train_data.document.isnull()][:5]

In [ ]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'] = test_data['document'].replace('', np.nan) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

# 3. Tokenization, token indexing, and padding

## 3.1 Tokenization

In [ ]:
# 불용어

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 한국어 토큰화를 위한 형태소 분석기 사용
# Open Korean Text (OKT) is an open source Korean tokenizer written in Scala, developed by Will Hohyon Ryu.

from konlpy.tag import Okt

okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

In [ ]:
# 토큰화와 불용어 제거 진행 (train set)

X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
train_data[:10]

In [ ]:
print(X_train[:10])

In [ ]:
# 토큰화와 불용어 제거 진행 (test set)

X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

## 3.2 Token indexing

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print('number of word index :', len(tokenizer.word_index))

In [ ]:
print(tokenizer.word_index)

In [ ]:
# 빈도수가 낮은 단어들은 자연어 처리에서 배제 (등장 빈도수가 3회 미만 확인)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 3회 미만인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
# draw Word Cloud

from wordcloud import WordCloud

fontpath = 'NanumGothic-Regular.ttf'

train_word_list = [x for xs in X_train for x in xs]

wc = WordCloud(
    width = 600,
    height = 600,
    max_words=20000,
    font_path = fontpath,
    background_color='white'
).generate(str(train_word_list))
plt.figure(figsize=(20, 10))
plt.imshow(wc)
plt.axis('off')

In [ ]:
# 3회 미만 단어 제거 이후 다시 tokenize

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:10])

In [ ]:
# save labels as np array

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

## 3.3 Padding (서로 다른 문장의 길이를 통일)

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 30
below_threshold_len(max_len, X_train)

In [ ]:
# pad

from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# 4. Model train

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# metric to watch during train
"""
my_metrics = [tf.keras.metrics.AUC(name='auc'),
                tf.keras.metrics.Precision(name='pr'),
                tf.keras.metrics.Recall(name='re'),
                tf.keras.metrics.BinaryAccuracy(name='ac')
                ]
"""
my_metrics = [
                tf.keras.metrics.BinaryAccuracy(name='ac')
                ]

In [ ]:
# early stopping metric

my_monitor = "val_ac"
#my_monitor = "val_re"
#my_monitor = "val_auc"

In [ ]:
# early stopping and model check point options

es = tf.keras.callbacks.EarlyStopping(monitor=my_monitor, mode='max', patience=15, restore_best_weights=True, verbose=1)
mc = tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor=my_monitor, mode='max', verbose=1, save_best_only=True)

In [ ]:
# compile model and fit

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=my_metrics)

history = model.fit(X_train, y_train, epochs=50, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

# 5. Predictions

In [ ]:
loaded_model = load_model('best_model.keras')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = loaded_model.predict(pad_new) # 예측
    score = score[0][0]

    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')

In [ ]:
sentiment_predict('와 쩐다 정말 세계관 최강자들의 영화다')